In [1]:
%%capture
!pip install transformers
!pip install nltk
#!pip install tensorflow
#!pip isntall tensoflow-gpu

In [2]:
import nltk 

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import pandas as pd
import tensorflow as tf
import re
import nltk
import string
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras import activations, optimizers, losses
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import numpy as np


In [4]:
# reduce dataset size for testing
# implement predict -save and load model
# Add Roberta with fine tuning
# Add BERT with fine tuning
# Calculate f1 and precision

In [5]:
print(tf.__version__)

2.3.2


In [6]:
class MyTriangularSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self,initial_learning_rate, end_learning_rate,cycle_steps,warmup_fraction = 0.1):
    self.initial_learning_rate = initial_learning_rate
    self.end_learning_rate = end_learning_rate
    self.cycle_steps = cycle_steps
    self.warmup_fraction = warmup_fraction

  def __call__(self, step):
    if step <= self.cycle_steps * self.warmup_fraction:
       unit_cycle = step * 1 / (self.cycle_steps * self.warmup_fraction)
    elif step <= self.cycle_steps:
      unit_cycle = (self.cycle_steps - step) * 1 / (self.cycle_steps * (1 - self.warmup_fraction))
    else:
      unit_cycle = 0 

In [7]:
from transformers.optimization_tf import WarmUp

# Extract Data

In [8]:
train = pd.read_csv('train.csv')

def get_samples(ratio,train):
  great_than2_classes = train['event'].value_counts()[train['event'].value_counts() >2].index
  train = train[train['event'].isin(great_than2_classes.to_list())]
  train_samples, _ = train_test_split(train,train_size=ratio,random_state=42,stratify=train['event'])
  print("nb classes",train_samples['event'].nunique())
  print("nb oservations:",train_samples.shape)
  print(train_samples['event'].value_counts())

  return train_samples

#get_samples(ratio = 0.05, train=train)

In [9]:
def get_data(is_sample=None,ratio=None,is_test_split=False):
  
    train = pd.read_csv('train.csv')
    if is_sample:
      train = get_samples(ratio = ratio, train=train)
    great_than2_classes = train['event'].value_counts()[train['event'].value_counts() >7].index 
    train_filter = train[train['event'].isin(great_than2_classes.to_list())]

    X = train_filter['text']
    y = train_filter['event']

    print(f"X.shape {X.shape} y.shape : {y.shape}")

    X_train_valid,X_test,y_train_valid, y_test = train_test_split(X,y,train_size=0.9,random_state=42,stratify=y)
    
    if is_test_split:
      X_train,X_valid,y_train,y_valid = train_test_split(X_train_valid,y_train_valid,train_size=0.8,random_state=42,stratify=y_train_valid)

    if is_test_split :
      print(f"X_train shape {X_train.shape} y_train shape : {y_train.shape}")
      print(f"X_valid shape {X_valid.shape} y_valid shape : {y_valid.shape}")
      print(f"X_test shape {X_test.shape} y_test shape : {y_test.shape}")

      return {
          'train': (X_train,y_train),
          'valid': (X_valid,y_valid),
          'test': (X_test,y_test)
      }

    else:
      print(f"X_train shape {X_train_valid.shape} y_train shape : {y_train_valid.shape}")
      print(f"X_valid shape {X_test.shape} y_valid shape : {y_test.shape}")


      return {
          'train': (X_train_valid,y_train_valid),
          'valid': (X_test,y_test),
      }


In [10]:
pd.options.display.max_colwidth = 3100
data = get_data(is_sample=False,ratio=1)
X_train, y_train = data['train']
X_valid,y_valid = data['valid']
#X_test,y_test = data['test']

X.shape (153938,) y.shape : (153938,)
X_train shape (138544,) y_train shape : (138544,)
X_valid shape (15394,) y_valid shape : (15394,)


# Preprocess Data

In [11]:
import numpy as np
print('classes in train :',len(np.unique(y_train)))
print('classes in valid :',len(np.unique(y_valid)))

CLASSES = y_train.unique().tolist()
print(CLASSES)

classes in train : 42
classes in valid : 42
[42, 62, 71, 55, 43, 73, 63, 31, 60, 41, 70, 64, 12, 99, 11, 53, 72, 13, 26, 52, 66, 61, 67, 27, 24, 51, 32, 23, 78, 44, 25, 79, 50, 40, 65, 49, 22, 69, 21, 20, 45, 54]


In [12]:

def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """

    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)

    # convert text to lowercase
    text = text.strip().lower()

    # remove all non-ASCII characters:
    text = re.sub(r'[^\x00-\x7f]', r'', text)

    # replace punctuation characters with spaces
    filters = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = " ".join(text.split())
    return text

In [13]:
def remove_useless_words(text,useless_words):
    sentence = [word for word in word_tokenize(text)]
    sentence_stop = [word for word in sentence if word not in useless_words]

    text = " ".join(sentence_stop)

    return text

In [14]:
def preprocess_data(X):
    """ Preprocess : cleaning and remove stop words"""

    X = X.apply(lambda x: re.sub(r'\d+', '', x))
    X = X.apply(lambda x: clean_text(x))

    stopwords = nltk.corpus.stopwords.words('english')
    useless_words = stopwords + list(string.punctuation) + ['yom', 'yof', 'yowm', 'yf', 'ym', 'yo']
    # print("useless word : ",useless_words)
    X = X.apply(lambda x: remove_useless_words(x,useless_words))

    return X

In [15]:
X_train_processed = preprocess_data(X_train)
X_valid_processed = preprocess_data(X_valid)

print("after preprocessing...")
print(X_train_processed.head(5))

after preprocessing...
59414     walking steps job missed step slipped fell hit rt knee felt pop dx quad tendon rupture
48134                         c foot injury p back foot hit linen cart work dx right foot injury
136481                              paramedic twisted ankle lifting patient work dx ankle sprain
54909                                      unhooking iv line got iv fluid splashed face occ expo
18265                                  shoulder injury fell ladder carrying bundle shingles work
Name: text, dtype: object


Save Processed data as TFRecords

# Tokenize and Encode Train and Test data

Test and Preview Tokenizer in transformers

In [16]:
from transformers import  DistilBertTokenizerFast,BertTokenizerFast,AutoTokenizer

MAX_LEN = 45
MODEL_NAME = 'distilbert-base-uncased'
#tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)


#narrative = 'r wrist lac cut wrist yest scraped wrist piece rusty metal happened working camp'



#MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

narratives = [X_train_processed.iloc[2],X_valid_processed.iloc[2]]
inputs = tokenizer(narratives, max_length=MAX_LEN, truncation=True, padding='max_length')

print(f"max_len:{MAX_LEN}")
print(f'narrative: \'{narratives}\'')
print(f'input ids: {inputs["input_ids"]}')
print(f'attention mask: {inputs["attention_mask"]}')

max_len:45
narrative: '['paramedic twisted ankle lifting patient work dx ankle sprain', 'working c metal grinder wearing gloves kicked back hittinghand lac']'
input ids: [[101, 11498, 7583, 2594, 6389, 10792, 8783, 5776, 2147, 1040, 2595, 10792, 11867, 21166, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2551, 1039, 3384, 23088, 2121, 4147, 11875, 6476, 2067, 7294, 11774, 18749, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [17]:
def construct_encodings(x, tkzr, max_len, trucation=True, padding=True):
    return tkzr(x, max_length=max_len, truncation=trucation, padding=padding)

x_train = X_train_processed.to_list()
x_valid = X_valid_processed.to_list()

#train_encodings = construct_encodings(x_train, tokenizer, max_len=MAX_LEN)
#valid_encodings = construct_encodings(x_valid, tokenizer, max_len=MAX_LEN)

train_encodings = tokenizer(x_train, max_length=MAX_LEN, truncation=True, padding='max_length',return_tensors='tf')
valid_encodings = tokenizer(x_valid, max_length=MAX_LEN, truncation=True, padding='max_length',return_tensors='tf')


In [18]:
print(MAX_LEN)
print(len(train_encodings['input_ids'][0]))
print(len(valid_encodings['input_ids'][0]))

45
45
45


In [19]:
from sklearn.preprocessing import LabelEncoder

print("Encoding Labels .....")
encoder = LabelEncoder()
encoder.fit(y_train)
y_train_encode = np.asarray(encoder.transform(y_train))
y_valid_encode = np.asarray(encoder.transform(y_valid))

Encoding Labels .....


In [20]:
print('classes in train :',len(np.unique(y_train_encode)))
print('classes in valid :',len(np.unique(y_valid_encode)))

classes in train : 42
classes in valid : 42


## Create TF Dataset

In [21]:
%%time
def construct_tfdataset(encodings, y=None):
    if y is not None:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))
    
train_tfdataset = construct_tfdataset(train_encodings, y_train_encode)
valid_tfdataset = construct_tfdataset(valid_encodings, y_valid_encode)

CPU times: user 2.86 ms, sys: 155 µs, total: 3.02 ms
Wall time: 2.9 ms


In [22]:
print(train_tfdataset)
print(valid_tfdataset)


<TensorSliceDataset shapes: ({input_ids: (45,), attention_mask: (45,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>
<TensorSliceDataset shapes: ({input_ids: (45,), attention_mask: (45,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>


In [23]:
#N_EPOCHS=3
#BATCH_SIZE = 16 
#steps_per_epoch = len(X_train) // BATCH_SIZE

#train_tfdataset = train_tfdataset.repeat(N_EPOCHS * steps_per_epoch)
#train_tfdataset = train_tfdataset.prefetch(tf.data.AUTOTUNE)
#train_tfdataset = train_tfdataset.batch(BATCH_SIZE)

In [24]:
#valid_tfdataset = valid_tfdataset.repeat(N_EPOCHS * steps_per_epoch)
#valid_tfdataset = valid_tfdataset.prefetch(tf.data.AUTOTUNE)
#valid_tfdataset = valid_tfdataset.batch(BATCH_SIZE)

In [25]:
print(train_tfdataset)
print(valid_tfdataset)

<TensorSliceDataset shapes: ({input_ids: (45,), attention_mask: (45,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>
<TensorSliceDataset shapes: ({input_ids: (45,), attention_mask: (45,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>


# Fine Tune DistilBERT

In [26]:
from transformers import  TFDistilBertForSequenceClassification
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from transformers.optimization_tf import WarmUp, AdamWeightDecay

class DistillBERT:
    def __init__(self, params):
        self.num_labels = params['num_labels']
        self.max_len = params['max_len']
        self.learning_rate = params['learning_rate']
        self.num_records = params['num_records']
        self.batch_size = params['batch_size']
        self.epochs = params['epochs']
        
    def build(self):
        
        freeze_bert_layer = False
        
        # Load model and tokenizer
        transformer_model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=self.num_labels)
        input_ids = tf.keras.Input(shape = (self.max_len,),name='input_ids',dtype='int32')
        attention_mask = tf.keras.Input(shape = (self.max_len,),name='attention_mask',dtype='int32')
        
        embbeding_layer = transformer_model.distilbert(input_ids,attention_mask=attention_mask)[0]
        #X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(embbeding_layer)
        X = tf.keras.layers.Conv1D(512, 3, activation='relu')(embbeding_layer)
        X = tf.keras.layers.GlobalMaxPool1D()(X)
        X = tf.keras.layers.Dense(256, activation='relu')(X)
        X = tf.keras.layers.Dropout(0.2)(X)
        X = tf.keras.layers.Dense(self.num_labels, activation='softmax')(X)
        
        model = tf.keras.Model(inputs=[input_ids,attention_mask], outputs = X)
        
        for layer in model.layers[:3]:
            layer.trainable = not freeze_bert_layer

        num_train_steps = (self.num_records // self.batch_size) * self.epochs
        decay_schedule_fn = PolynomialDecay(
            initial_learning_rate=self.learning_rate,
            end_learning_rate=0.,
            decay_steps=num_train_steps
            )
        
        warmup_steps  = num_train_steps * 0.1
        warmup_schedule = WarmUp(self.learning_rate,decay_schedule_fn,warmup_steps)

          # fine optimizer and loss
        #optimizer = tf.keras.optimizers.Adam(learning_rate=warmup_schedule,epsilon=1e-06)

        optimizer = AdamWeightDecay(learning_rate=warmup_schedule, weight_decay_rate=0.01, epsilon=1e-6)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= False)
        metrics = ['acc']
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

        return model

In [27]:
LEARNING_RATE = 5e-5
params = {
        'num_labels': len(CLASSES),
        'max_len': MAX_LEN,
        'learning_rate': LEARNING_RATE,
        'num_records':len(X_train),
        'batch_size':16,
        'epochs':1
    }
  
model = DistillBERT(params).build()   
print(model.summary())


[2021-08-19 22:37:22.537 tensorflow-2-3-gpu--ml-g4dn-xlarge-c85184389676cdfa7bdf06745c9b:104 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-19 22:37:22.563 tensorflow-2-3-gpu--ml-g4dn-xlarge-c85184389676cdfa7bdf06745c9b:104 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 45)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 45)]         0                                            
__________________________________________________________________________________________________
distilbert (TFDistilBertMainLay TFBaseModelOutput(la 66362880    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 43, 512)      1180160     distilbert[0][0]      

In [28]:
%%time
N_EPOCHS=1
BATCH_SIZE = 16 
model.fit(x=train_tfdataset.batch(BATCH_SIZE),
          batch_size=BATCH_SIZE, 
          epochs=N_EPOCHS,
          validation_data=valid_tfdataset.batch(BATCH_SIZE))

8659/8659 [==============================] - 833s 96ms/step - loss: 0.7041 - acc: 0.7893 - val_loss: 0.4528 - val_acc: 0.8509
CPU times: user 9min 16s, sys: 3min 17s, total: 12min 33s
Wall time: 14min 8s


In [29]:
train_tfdataset
valid_tfdataset

<TensorSliceDataset shapes: ({input_ids: (45,), attention_mask: (45,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>

In [30]:
%%time
model.evaluate(valid_tfdataset.batch(BATCH_SIZE), batch_size=BATCH_SIZE,return_dict=True)

963/963 [==============================] - 25s 26ms/step - loss: 0.4528 - acc: 0.8509
CPU times: user 17.4 s, sys: 3 s, total: 20.4 s
Wall time: 25.5 s


{'loss': 0.45275986194610596, 'acc': 0.8508509993553162}

In [ ]:
model.save("distillbert.h5")

# Train DistilBERT

In [ ]:
N_EPOCHS = 5
BATCH_SIZE = 16 
print(MODEL_NAME)

model2 = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(CLASSES))
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model2.summary()

distilbert-base-uncased


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_376', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use 

Model: "tf_distil_bert_for_sequence_classification_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  21532     
_________________________________________________________________
dropout_376 (Dropout)        multiple                  0         
Total params: 66,975,004
Trainable params: 66,975,004
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time

model2.fit(train_tfdataset.shuffle(1000).batch(BATCH_SIZE), batch_size=BATCH_SIZE, epochs=N_EPOCHS,validation_data=valid_tfdataset.batch(BATCH_SIZE))

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

# Train Base BERT

In [31]:
from transformers import TFRobertaForSequenceClassification,TFBertForSequenceClassification
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from transformers.optimization_tf import WarmUp, AdamWeightDecay
class BaseBERT:
    def __init__(self, params):
        self.num_labels = params['num_labels']
        self.max_len = params['max_len']
        self.learning_rate = params['learning_rate']
        self.num_records = params['num_records']
        self.batch_size = params['batch_size']
        self.epochs = params['epochs']
        
    def build(self):
        
        freeze_bert_layer = False
        
        # Load model and tokenizer
        transformer_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=self.num_labels)
        input_ids = tf.keras.Input(shape = (self.max_len,),name='input_ids',dtype='int32')
        attention_mask = tf.keras.Input(shape = (self.max_len,),name='attention_mask',dtype='int32')
        
        embbeding_layer = transformer_model.bert(input_ids,attention_mask=attention_mask)[0]
        #X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(embbeding_layer)
        X = tf.keras.layers.Conv1D(512, 3, activation='relu')(embbeding_layer)
        X = tf.keras.layers.GlobalMaxPool1D()(X)
        X = tf.keras.layers.Dense(256, activation='relu')(X)
        X = tf.keras.layers.Dropout(0.2)(X)
        X = tf.keras.layers.Dense(self.num_labels, activation='softmax')(X)
        
        model = tf.keras.Model(inputs=[input_ids,attention_mask], outputs = X)
        
        for layer in model.layers[:3]:
            layer.trainable = not freeze_bert_layer

        num_train_steps = (self.num_records // self.batch_size) * self.epochs
        decay_schedule_fn = PolynomialDecay(
            initial_learning_rate=self.learning_rate,
            end_learning_rate=0.,
            decay_steps=num_train_steps
            )
        
        warmup_steps  = num_train_steps * 0.1
        warmup_schedule = WarmUp(self.learning_rate,decay_schedule_fn,warmup_steps)

          # fine optimizer and loss
        #optimizer = tf.keras.optimizers.Adam(learning_rate=warmup_schedule,epsilon=1e-06)

        optimizer = AdamWeightDecay(learning_rate=warmup_schedule, weight_decay_rate=0.01, epsilon=1e-6)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= False)
        metrics = ['acc']
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

        return model

In [32]:
LEARNING_RATE = 5e-5
params = {
        'num_labels': len(CLASSES),
        'max_len': MAX_LEN,
        'learning_rate': LEARNING_RATE,
        'num_records':len(X_train),
        'batch_size':16,
        'epochs':1
    }
  
model4 = BaseBERT(params).build()   
print(model4.summary())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 45)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 45)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 43, 512)      1180160     bert[0][0]            

In [33]:
%%time
N_EPOCHS=1
BATCH_SIZE = 16 
model4.fit(x=train_tfdataset.shuffle(len(X_train)).batch(BATCH_SIZE),
           batch_size=BATCH_SIZE,
           epochs=N_EPOCHS,
           validation_data=valid_tfdataset.batch(BATCH_SIZE))

8659/8659 [==============================] - 1519s 175ms/step - loss: 0.7114 - acc: 0.7879 - val_loss: 0.4517 - val_acc: 0.8508
CPU times: user 17min 3s, sys: 6min 8s, total: 23min 12s
Wall time: 25min 33s


In [56]:
from transformers import TFRobertaForSequenceClassification,TFBertForSequenceClassification

MODEL_NAME = "bert-base-uncased"

N_EPOCHS = 4
BATCH_SIZE = 16 

print(MODEL_NAME)

model3 = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(CLASSES),output_attentions = False,output_hidden_states= False )
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model3.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model3.summary()


bert-base-uncased


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_309 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  32298     
Total params: 109,514,538
Trainable params: 109,514,538
Non-trainable params: 0
_________________________________________________________________


In [28]:
%%time

model3.fit(train_tfdataset.shuffle(1000).batch(BATCH_SIZE), batch_size=BATCH_SIZE, epochs=N_EPOCHS,validation_data=valid_tfdataset.batch(BATCH_SIZE))



Epoch 1/4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

Using the fine-tuned model to predict new samples

In [ ]:
def distilBert_create_predictor(model, model_name, max_len):
  tkzr = DistilBertTokenizerFast.from_pretrained(model_name)
  def predict_proba(text):
      x = [text]
    
      encodings =  tkzr(x, max_length=max_len, truncation=True, padding='max_length',return_tensors='tf')
      tfdataset = construct_tfdataset(encodings)
      tfdataset = tfdataset.batch(1)

      preds = model.predict(tfdataset)
      preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()
      return preds[0][0]
    
  return predict_proba

clf = distilBert_create_predictor(model, MODEL_NAME, MAX_LEN)
print(clf('reports sus laceration rt palm lost footing whilecoming ladder dx palm laceration'))

0.023528205


In [ ]:
from transformers import BertTokenizerFast

def bert_create_predictor(model, model_name, max_len):
  tkzr = BertTokenizerFast.from_pretrained(model_name)
  def predict_proba(text):
      x = [text]
    
      encodings =  tkzr(x, max_length=max_len, truncation=True, padding='max_length',return_tensors='tf')
      tfdataset = construct_tfdataset(encodings)
      tfdataset = tfdataset.batch(1)

      preds = model.predict(tfdataset)
      preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()
      print(preds[0])
      return preds[0][0]
    
  return predict_proba

clf = bert_create_predictor(model, MODEL_NAME, MAX_LEN)
print(clf('reports sus laceration rt palm lost footing whilecoming ladder dx palm laceration'))

45
43
0.023528205
